In [1]:
# Run this script only when you haven't downloaded data yet. 
# !./prepare_input.sh

noaa_weather_qclcd_documentation.pdf: Downloaded 85KB of 85KB to ./input
weather.csv.zip: Downloaded 74KB of 74KB to ./input
spray.csv.zip: Downloaded 133KB of 133KB to ./input
train.csv.zip: Downloaded 122KB of 122KB to ./input
sampleSubmission.csv.zip: Downloaded 244KB of 244KB to ./input
test.csv.zip: Downloaded 1MB of 1MB to ./input
mapdata_copyright_openstreetmap_contributors.rds: Downloaded 2MB of 2MB to ./input
mapdata_copyright_openstreetmap_contributors.txt.zip: Downloaded 3MB of 3MB to ./input
west_nile.zip: Downloaded 8MB of 8MB to ./input
Archive:  spray.csv.zip
  inflating: spray.csv               

Archive:  west_nile.zip
   creating: west_nile/
  inflating: west_nile/.DS_Store     
   creating: __MACOSX/
   creating: __MACOSX/west_nile/
  inflating: __MACOSX/west_nile/._.DS_Store  
   creating: west_nile/input/
  inflating: west_nile/input/mapdata_copyright_openstreetmap_contributors.rds  
   creating: __MACOSX/west_nile/input/
  inflating: __MACOSX/west_nile/input/._map

In [1]:
import pandas as pd
import numpy as np
import lib.Model as md

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
raw_train_data = pd.read_csv('./input/train.csv')
raw_test_data = pd.read_csv('./input/test.csv')

weather_data = pd.read_csv('./input/weather.csv')
spray_data = pd.read_csv('./input/spray.csv')

In [3]:
def dummy_species(data): # TODO 高速化
    # 蚊の種類をone-hot形式にする
    sp_keys = {'CULEX ERRATICUS' , 'CULEX PIPIENS', 'CULEX RESTUANS',
 'CULEX SALINARIUS', 'CULEX TARSALIS', 'CULEX TERRITANS' , 'CULEX PIPIENS/RESTUANS'}
    sp_map = {'CULEX ERRATICUS' : np.array([1, 0, 0, 0, 0, 0]),
                        'CULEX PIPIENS' : np.array([0, 1, 0, 0 , 0, 0]),
                        'CULEX RESTUANS' : np.array([0, 0, 1, 0, 0, 0]),
                        'CULEX SALINARIUS' : np.array([0, 0, 0, 1, 0, 0]),
                        'CULEX TARSALIS' : np.array([0, 0, 0, 0, 1, 0]),
                        'CULEX TERRITANS' : np.array([0, 0, 0, 0, 0, 1]),
                        'CULEX PIPIENS/RESTUANS' : np.array([0, 1, 1, 0, 0, 0])}
    dummies = np.empty((0, len(sp_keys)-1), int)
    for i in data:
        if i not in sp_keys:
            dummies = np.append(dummies, [np.zeros(len(sp_keys)-1)], axis=0)
        else:
            dummies = np.append(dummies, [np.array(sp_map[i])], axis=0)
    return pd.DataFrame(dummies)

def dummy_trap(data): # TODO 高速化
    # trapの種類をone-hot形式にする
    trap_keys = list(set(raw_train_data['Trap']))
    trap_dummy = pd.get_dummies(trap_keys)
    dummies = np.empty((0, len(trap_keys)), int)
    for i in data:
            if i not in trap_keys:
                dummies = np.append(dummies, [np.zeros(len(trap_keys))], axis=0)
            else:
                dummies = np.append(dummies, [np.array(trap_dummy[i])], axis=0)
    return pd.DataFrame(dummies)
                
def preprocessing(data, weather, spray):
    # pandasのデータで前処理をしてnumpy配列を返す
    
    # 天気情報の前処理
    formated_weather = weather.drop_duplicates(subset='Date').drop(['Station', 'CodeSum', 'Depth', 'Water1'], axis=1)
    formated_weather = formated_weather.convert_objects(convert_numeric=True).fillna(0) # TODO 欠損値の扱い

    # TODO スプレーデータの前処理
    
    # メインデータの前処理
    #if 'NumMosquitos' in data: # TODO 重複データの扱い
    #    data = data.drop_duplicates(subset=['Date', 'Species', 'Latitude', 'Longitude', 'Trap'])
    
    species = dummy_species(data['Species'])
    trap = dummy_trap(data['Trap'])

    merged_data = pd.merge(data, formated_weather, on='Date')
    merged_data = pd.merge(merged_data, species, right_index=True, left_index=True)
    merged_data = pd.merge(merged_data, trap, right_index=True, left_index=True)

    if 'NumMosquitos' in merged_data:
        preprocessed_data = merged_data.drop(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'AddressAccuracy', 'NumMosquitos', 'WnvPresent'], axis=1)
        return np.array(preprocessed_data), np.array(data['NumMosquitos']), np.array(data['WnvPresent'])
    else:
        preprocessed_data = merged_data.drop(['Id','Date', 'Address', 'Species', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'AddressAccuracy'], axis=1)
        return np.array(preprocessed_data), np.array([]), np.array([])

In [4]:
print("Make training data")
train_data, num_mosquitos, label = preprocessing(raw_train_data, weather_data, spray_data)
print("Make test data")
test_data, tmp1, tmp2 = preprocessing(raw_test_data, weather_data, spray_data)

Make training data


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:37: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


Make test data


In [5]:
# SVM
model_svm = md.SVM(optimization=True)
model_svm.fit(train_data, label)
result_svm = model_svm.predict(test_data)

In [6]:
# RandomForest
model_rf = md.RandomForest()
model_rf.fit(train_data, label)
result_rf = model_rf.predict(test_data)

In [7]:
print(result_rf.shape)

(116293,)


In [8]:
# GLM
model_glm = md.GLM()
model_glm.fit(train_data, label, num_mosquitos)
result_glm = model_glm.predict(test_data)

100 [  3.67046564e-04   9.32039341e-04   4.89315058e-04   3.21366698e-04
   7.33068821e-04   2.90070379e-04   6.59425567e-04   6.49829006e-04
   1.64727383e-04   7.94534994e-05   4.50434634e-04   7.53705812e-04
   3.33082120e-04   5.81981741e-04   5.21427721e-04   4.26245789e-04
   3.02388211e-05   3.46446860e-04   6.24434134e-04   4.06327027e-04
   5.14730985e-04   4.30708759e-04   2.10649909e-04   4.51453675e-07
   8.69238853e-04   7.54268854e-05   6.33440540e-04   2.21677503e-04
   5.42397569e-04   1.61869246e-05   6.71090781e-04   2.54250259e-04
   2.00172345e-04   8.35560710e-04   7.97573157e-04   6.97587779e-04
   5.33506815e-04   7.27190690e-04   6.92578469e-04   6.31231791e-04
   2.55590665e-04   2.90145710e-04   8.68033495e-04   9.22778173e-04
   8.60009259e-04   6.84980237e-04   3.55147462e-04   9.76615272e-04
   5.35981129e-04   5.55072500e-04   2.92121958e-04   2.02764885e-04
   7.95117193e-04   7.48430979e-04   3.73768494e-04   6.25093734e-04
   3.02049722e-04   6.00246708

400 [  3.67046564e-04   9.32039341e-04   4.89315058e-04   3.21366698e-04
   7.33068821e-04   2.90070379e-04   6.59425567e-04   6.49829006e-04
   1.64727383e-04   7.94534994e-05   4.50434634e-04   7.53705812e-04
   3.33082120e-04   5.81981741e-04   5.21427721e-04   4.26245789e-04
   3.02388211e-05   3.46446860e-04   6.24434134e-04   4.06327027e-04
   5.14730985e-04   4.30708759e-04   2.10649909e-04   4.51453675e-07
   8.69238853e-04   7.54268854e-05   6.33440540e-04   2.21677503e-04
   5.42397569e-04   1.61869246e-05   6.71090781e-04   2.54250259e-04
   2.00172345e-04   8.35560710e-04   7.97573157e-04   6.97587779e-04
   5.33506815e-04   7.27190690e-04   6.92578469e-04   6.31231791e-04
   2.55590665e-04   2.90145710e-04   8.68033495e-04   9.22778173e-04
   8.60009259e-04   6.84980237e-04   3.55147462e-04   9.76615272e-04
   5.35981129e-04   5.55072500e-04   2.92121958e-04   2.02764885e-04
   7.95117193e-04   7.48430979e-04   3.73768494e-04   6.25093734e-04
   3.02049722e-04   6.00246708

700 [  3.67046564e-04   9.32039341e-04   4.89315058e-04   3.21366698e-04
   7.33068821e-04   2.90070379e-04   6.59425567e-04   6.49829006e-04
   1.64727383e-04   7.94534994e-05   4.50434634e-04   7.53705812e-04
   3.33082120e-04   5.81981741e-04   5.21427721e-04   4.26245789e-04
   3.02388211e-05   3.46446860e-04   6.24434134e-04   4.06327027e-04
   5.14730985e-04   4.30708759e-04   2.10649909e-04   4.51453675e-07
   8.69238853e-04   7.54268854e-05   6.33440540e-04   2.21677503e-04
   5.42397569e-04   1.61869246e-05   6.71090781e-04   2.54250259e-04
   2.00172345e-04   8.35560710e-04   7.97573157e-04   6.97587779e-04
   5.33506815e-04   7.27190690e-04   6.92578469e-04   6.31231791e-04
   2.55590665e-04   2.90145710e-04   8.68033495e-04   9.22778173e-04
   8.60009259e-04   6.84980237e-04   3.55147462e-04   9.76615272e-04
   5.35981129e-04   5.55072500e-04   2.92121958e-04   2.02764885e-04
   7.95117193e-04   7.48430979e-04   3.73768494e-04   6.25093734e-04
   3.02049722e-04   6.00246708

1000 [  3.65066887e-04   9.18706256e-04   4.93961678e-04   3.22793946e-04
   7.40863903e-04   2.89900327e-04   6.65593148e-04   6.56162316e-04
   1.65219143e-04   7.93243530e-05   4.60508136e-04   7.63966916e-04
   3.31587523e-04   5.83701462e-04   5.72984308e-04   4.24289365e-04
   3.00734404e-05   3.46284994e-04   6.23426141e-04   4.06199019e-04
   5.13090855e-04   4.28868170e-04   2.09563003e-04   4.49581476e-07
   8.70621765e-04   7.50494950e-05   6.32545689e-04   2.21146514e-04
   5.41807016e-04   1.62320297e-05   6.73081232e-04   2.54314396e-04
   1.99997740e-04   8.37744113e-04   7.97707763e-04   6.94919622e-04
   5.33457728e-04   7.23851206e-04   6.94186393e-04   6.28009132e-04
   2.54552562e-04   2.89764825e-04   8.64137973e-04   9.18537748e-04
   8.58529184e-04   6.85763023e-04   3.55480231e-04   9.77986114e-04
   5.33908708e-04   5.56250984e-04   2.92568676e-04   2.02357037e-04
   7.97287939e-04   7.46134174e-04   3.73319784e-04   6.24802594e-04
   3.03036333e-04   5.9743815

1300 [  3.68486682e-04   9.15179575e-04   4.94264827e-04   3.21602274e-04
   7.40568057e-04   2.91428366e-04   6.66071762e-04   6.56471719e-04
   1.64607951e-04   7.98014650e-05   4.63369914e-04   7.86205117e-04
   3.33831910e-04   5.82273569e-04   5.30781976e-04   4.27529238e-04
   3.03145570e-05   3.48198044e-04   6.27333534e-04   4.03187029e-04
   5.16667985e-04   4.32084453e-04   2.09693944e-04   4.50468503e-07
   8.70344408e-04   7.49885868e-05   6.33768123e-04   2.19967299e-04
   5.41872283e-04   1.60752870e-05   6.61661233e-04   2.54244807e-04
   2.00006866e-04   8.37807812e-04   7.97503704e-04   6.96310232e-04
   5.33354960e-04   7.22453272e-04   6.94137763e-04   6.28589978e-04
   2.55066730e-04   2.90282539e-04   8.65758527e-04   9.20110301e-04
   8.60595074e-04   6.85513262e-04   3.55351404e-04   9.77647397e-04
   5.32153090e-04   5.56156687e-04   2.92472379e-04   2.01185824e-04
   7.97398744e-04   7.42800575e-04   3.73379531e-04   6.24748856e-04
   3.00133126e-04   5.9842159

...

8500 [  3.49395046e-04   4.69379420e-04   2.87558364e-04   2.30989029e-04
   6.74703466e-04   2.68834755e-04   7.50836807e-04   7.77438895e-04
   1.30176578e-04   9.45125319e-05   5.63198138e-04   3.04364994e-03
   4.98504518e-04   6.24415184e-04   1.22706679e-03   6.96836917e-04
   3.37184936e-05   3.95214017e-04   7.84071976e-04   2.61071459e-04
   6.47024608e-04   1.47896953e-04   2.22060326e-04   4.79431059e-07
   1.55280564e-04   8.49852752e-05   7.10046217e-04   1.51504262e-04
   4.59497653e-04   1.45243310e-05   2.90020960e-04   1.89375039e-04
   2.41435184e-04   5.28623847e-04   6.61024966e-04   6.71674033e-04
   3.75440315e-04   7.75203617e-04   6.38085809e-04   5.89305025e-04
   2.39988980e-04   3.79489166e-04   1.06975399e-03   9.67485984e-04
   1.26817840e-03   7.92708846e-04   3.04631292e-04   1.00822547e-03
   3.93654927e-04   6.44961704e-04   2.06101115e-04   2.99735563e-04
   3.69077885e-04   4.41303876e-04   3.29146578e-04   7.11683454e-04
   1.07788162e-04   4.0298287

8800 [  3.47841332e-04   4.05296019e-04   2.68844562e-04   2.34858467e-04
   6.49659240e-04   2.60883372e-04   7.69295118e-04   7.90078714e-04
   1.24295332e-04   9.68467484e-05   5.67427711e-04   3.32207730e-03
   5.63901012e-04   6.26731705e-04   1.29584524e-03   7.38276125e-04
   3.37966536e-05   3.96161216e-04   8.08121229e-04   2.41241277e-04
   6.64913249e-04   1.16584124e-04   2.21548992e-04   4.58860185e-07
   6.62279367e-05   8.49336097e-05   7.18062135e-04   1.41908832e-04
   4.43015675e-04   1.39745002e-05   2.50875205e-04   1.81562475e-04
   2.49361221e-04   4.92783303e-04   6.39181821e-04   6.76592382e-04
   3.52934349e-04   7.83348440e-04   6.32725459e-04   5.73445408e-04
   2.39575268e-04   3.91388721e-04   1.10880298e-03   9.70469002e-04
   1.33722946e-03   8.06367982e-04   2.95075577e-04   1.00509526e-03
   3.73993616e-04   6.59898726e-04   1.94592972e-04   3.17409869e-04
   2.33985114e-04   4.02199729e-04   3.26720652e-04   7.28480319e-04
   7.40730907e-05   3.8296664

9100 [  3.45764065e-04   3.52205187e-04   2.89613693e-04   2.22333529e-04
   6.45428688e-04   2.58978325e-04   7.74183764e-04   7.86323909e-04
   1.19540566e-04   9.89455346e-05   5.83007355e-04   3.54687414e-03
   5.87025332e-04   6.34678923e-04   1.37781805e-03   7.71112416e-04
   3.44615270e-05   4.01013234e-04   8.27464668e-04   2.24426772e-04
   6.76965023e-04   8.79127895e-05   2.23724224e-04   4.64036257e-07
  -1.38799001e-05   8.68182963e-05   7.14016232e-04   1.33034722e-04
   4.30126883e-04   1.38497341e-05   2.09395458e-04   1.77155060e-04
   2.51636832e-04   4.48431079e-04   6.25848372e-04   6.84758140e-04
   3.40791247e-04   7.90044234e-04   6.18954091e-04   5.71985251e-04
   2.40542935e-04   4.04578317e-04   1.13251159e-03   9.82939289e-04
   1.37139884e-03   8.17497909e-04   2.88675251e-04   1.00430461e-03
   3.57774004e-04   6.66239883e-04   1.83933595e-04   3.28204364e-04
   2.31098890e-04   3.63872025e-04   3.21463644e-04   7.41231131e-04
   5.60701505e-05   3.6083289

9400 [  3.41781514e-04   2.51210558e-04   2.23983228e-04   2.07334285e-04
   6.32329912e-04   2.49624676e-04   7.82534620e-04   8.11510775e-04
   1.11744942e-04   1.01441439e-04   5.99317411e-04   3.87912747e-03
   6.18495463e-04   6.43476390e-04   1.48318275e-03   8.13198570e-04
   3.49442337e-05   4.02211882e-04   8.50838713e-04   2.03373036e-04
   6.92645285e-04   3.60878076e-05   2.23432641e-04   4.65487936e-07
  -1.16139687e-04   8.76674650e-05   7.30441213e-04   1.23303772e-04
   4.11285252e-04   1.36910219e-05   1.91908549e-04   1.68053876e-04
   2.58103809e-04   4.21978670e-04   6.02714419e-04   6.92882895e-04
   3.20015639e-04   7.94436074e-04   5.99952835e-04   5.55437318e-04
   2.38402411e-04   4.21558234e-04   1.19350398e-03   1.00837161e-03
   1.43420370e-03   8.34174014e-04   2.79842406e-04   1.00160432e-03
   3.35097665e-04   6.83175767e-04   1.67851721e-04   3.46977703e-04
   1.49000166e-04   3.16607685e-04   3.19335343e-04   7.62008352e-04
   2.63161599e-05   3.2901905

9700 [  3.38868043e-04   1.44495812e-04   1.94923847e-04   1.91277138e-04
   6.11081319e-04   2.42833994e-04   8.07802149e-04   8.27097480e-04
   1.04535415e-04   1.05058787e-04   6.12588917e-04   4.24973866e-03
   6.56823443e-04   6.55425016e-04   1.60739273e-03   8.75446120e-04
   3.59464953e-05   4.07245800e-04   8.82225197e-04   1.76621152e-04
   7.14100440e-04  -6.12800911e-06   2.18677008e-04   4.64543731e-07
  -2.23752346e-04   8.91904433e-05   7.40402603e-04   1.10626329e-04
   3.87876744e-04   1.33788422e-05   1.23788862e-04   1.55304197e-04
   2.61668864e-04   3.57810716e-04   5.69330048e-04   7.01226899e-04
   2.95411018e-04   7.99618395e-04   5.84112651e-04   5.48330323e-04
   2.38178757e-04   4.40128878e-04   1.21604496e-03   1.00460482e-03
   1.51347435e-03   8.27397582e-04   2.86627403e-04   1.03008628e-03
   3.09688135e-04   7.24421873e-04   1.60638891e-04   3.66353778e-04
   4.74651395e-05   2.58340456e-04   3.11102636e-04   7.78736981e-04
  -6.77957544e-06   3.0116768

In [12]:
submitData_svm = pd.DataFrame.from_dict({"Id": raw_test_data['Id'], "WnvPresent": result_svm})
submitData_rf = pd.DataFrame.from_dict({"Id": raw_test_data['Id'], "WnvPresent": result_rf})
submitData_glm = pd.DataFrame.from_dict({"Id": raw_test_data['Id'], "WnvPresent": result_glm})

In [13]:
submitData_svm.to_csv('submission_svm.csv', index=0)
submitData_rf.to_csv('submission_rf.csv', index=0)
submitData_glm.to_csv('submission_glm.csv', index=0)

In [14]:
"""
        private    public
svm       0.515     0.510
rf        0.670     0.713
glm       0.474     0.488
"""

'\n        private    public\nsvm       0.515     0.510\nrf        0.670     0.713\nglm       0.474     0.488\n'